In [15]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy
import os

In [16]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [17]:
import numpy as np
DATA_PATH = "./data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [18]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [19]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [24]:
save_data_to_array()

NoBackendError: 

In [20]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=300, verbose=1, validation_data=(X_test, y_test_hot))

Train on 9261 samples, validate on 6175 samples
Epoch 1/300
9261/9261 [==============================] - 3s 304us/step - loss: 8.4758 - acc: 0.3537 - val_loss: 1.2390 - val_acc: 0.4509
Epoch 2/300
9261/9261 [==============================] - 2s 243us/step - loss: 1.2488 - acc: 0.4708 - val_loss: 1.0077 - val_acc: 0.5658
Epoch 3/300
9261/9261 [==============================] - 2s 258us/step - loss: 1.0441 - acc: 0.5624 - val_loss: 0.7822 - val_acc: 0.6954
Epoch 4/300
9261/9261 [==============================] - 2s 222us/step - loss: 0.9158 - acc: 0.6213 - val_loss: 0.7046 - val_acc: 0.7224
Epoch 5/300
9261/9261 [==============================] - 2s 203us/step - loss: 0.8227 - acc: 0.6616 - val_loss: 0.6707 - val_acc: 0.7504
Epoch 6/300
9261/9261 [==============================] - 2s 201us/step - loss: 0.7480 - acc: 0.6939 - val_loss: 0.5770 - val_acc: 0.7943
Epoch 7/300
9261/9261 [==============================] - 2s 204us/step - loss: 0.6988 - acc: 0.7182 - val_loss: 0.5688 - val_acc: 

9261/9261 [==============================] - 2s 192us/step - loss: 0.2165 - acc: 0.9179 - val_loss: 0.3232 - val_acc: 0.8920
Epoch 61/300
9261/9261 [==============================] - 2s 195us/step - loss: 0.2011 - acc: 0.9234 - val_loss: 0.2942 - val_acc: 0.9015
Epoch 62/300
9261/9261 [==============================] - 2s 191us/step - loss: 0.2049 - acc: 0.9228 - val_loss: 0.2940 - val_acc: 0.8988
Epoch 63/300
9261/9261 [==============================] - 2s 196us/step - loss: 0.2163 - acc: 0.9178 - val_loss: 0.3055 - val_acc: 0.8920
Epoch 64/300
9261/9261 [==============================] - 2s 196us/step - loss: 0.1985 - acc: 0.9255 - val_loss: 0.2977 - val_acc: 0.8973
Epoch 65/300
9261/9261 [==============================] - 2s 194us/step - loss: 0.1996 - acc: 0.9244 - val_loss: 0.2942 - val_acc: 0.8978
Epoch 66/300
9261/9261 [==============================] - 2s 194us/step - loss: 0.2090 - acc: 0.9199 - val_loss: 0.3275 - val_acc: 0.8886
Epoch 67/300
9261/9261 [=======================

9261/9261 [==============================] - 2s 265us/step - loss: 0.1494 - acc: 0.9473 - val_loss: 0.3303 - val_acc: 0.8991
Epoch 120/300
9261/9261 [==============================] - 2s 212us/step - loss: 0.1520 - acc: 0.9437 - val_loss: 0.3047 - val_acc: 0.9002
Epoch 121/300
9261/9261 [==============================] - 2s 249us/step - loss: 0.1386 - acc: 0.9499 - val_loss: 0.3113 - val_acc: 0.9059
Epoch 122/300
9261/9261 [==============================] - 3s 275us/step - loss: 0.1418 - acc: 0.9468 - val_loss: 0.3056 - val_acc: 0.9025
Epoch 123/300
9261/9261 [==============================] - 3s 292us/step - loss: 0.1319 - acc: 0.9508 - val_loss: 0.3566 - val_acc: 0.8983
Epoch 124/300
9261/9261 [==============================] - 2s 262us/step - loss: 0.1412 - acc: 0.9501 - val_loss: 0.3381 - val_acc: 0.9022
Epoch 125/300
9261/9261 [==============================] - 2s 259us/step - loss: 0.1359 - acc: 0.9492 - val_loss: 0.3314 - val_acc: 0.9022
Epoch 126/300
9261/9261 [================

9261/9261 [==============================] - 2s 226us/step - loss: 0.1199 - acc: 0.9596 - val_loss: 0.3579 - val_acc: 0.9020
Epoch 179/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.1142 - acc: 0.9603 - val_loss: 0.3561 - val_acc: 0.9023
Epoch 180/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.1149 - acc: 0.9595 - val_loss: 0.3474 - val_acc: 0.8998
Epoch 181/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.1176 - acc: 0.9579 - val_loss: 0.3420 - val_acc: 0.9004
Epoch 182/300
9261/9261 [==============================] - 2s 235us/step - loss: 0.1172 - acc: 0.9586 - val_loss: 0.3762 - val_acc: 0.8962
Epoch 183/300
9261/9261 [==============================] - 2s 228us/step - loss: 0.1163 - acc: 0.9594 - val_loss: 0.3335 - val_acc: 0.8964
Epoch 184/300
9261/9261 [==============================] - 2s 226us/step - loss: 0.1221 - acc: 0.9595 - val_loss: 0.3438 - val_acc: 0.9035
Epoch 185/300
9261/9261 [================

9261/9261 [==============================] - 2s 228us/step - loss: 0.1012 - acc: 0.9640 - val_loss: 0.3671 - val_acc: 0.9032
Epoch 238/300
9261/9261 [==============================] - 2s 227us/step - loss: 0.1076 - acc: 0.9619 - val_loss: 0.3712 - val_acc: 0.9011
Epoch 239/300
9261/9261 [==============================] - 2s 230us/step - loss: 0.1073 - acc: 0.9620 - val_loss: 0.3645 - val_acc: 0.8946
Epoch 240/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.1012 - acc: 0.9617 - val_loss: 0.3701 - val_acc: 0.9035
Epoch 241/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.1182 - acc: 0.9591 - val_loss: 0.3574 - val_acc: 0.9025
Epoch 242/300
9261/9261 [==============================] - 2s 226us/step - loss: 0.1076 - acc: 0.9626 - val_loss: 0.3679 - val_acc: 0.9035
Epoch 243/300
9261/9261 [==============================] - 2s 227us/step - loss: 0.1004 - acc: 0.9666 - val_loss: 0.3583 - val_acc: 0.9038
Epoch 244/300
9261/9261 [================

9261/9261 [==============================] - 2s 226us/step - loss: 0.0994 - acc: 0.9639 - val_loss: 0.3499 - val_acc: 0.9067
Epoch 297/300
9261/9261 [==============================] - 2s 226us/step - loss: 0.1036 - acc: 0.9634 - val_loss: 0.3785 - val_acc: 0.9020
Epoch 298/300
9261/9261 [==============================] - 2s 225us/step - loss: 0.0911 - acc: 0.9681 - val_loss: 0.3607 - val_acc: 0.9067
Epoch 299/300
9261/9261 [==============================] - 2s 226us/step - loss: 0.0921 - acc: 0.9692 - val_loss: 0.3930 - val_acc: 0.9032
Epoch 300/300
9261/9261 [==============================] - 2s 227us/step - loss: 0.0919 - acc: 0.9683 - val_loss: 0.3849 - val_acc: 0.9020


In [23]:
import socket
import numpy as np
from pygame import mixer
import time
import pyaudio
import 

'''
try:
        mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print ("Socket created")

except socket.error:
        print ("Socket connection failed")

port = 8888
try:
        mySock.connect(("192.168.43.195", port))
        print ("Connected to RPi!")
except socket.error:
        print ("Not able to connect")

'''        

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "out.wav"
while True:
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("* done recording")

        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        recording = AudioSegment.from_wav("./out.wav")
        recording+=20
        recording.export("./out.wav", format="wav")

        print("Before the sleep statement")
        mixer.init(frequency=44100)
        mixer.music.load("./out.wav")
        mixer.music.play()
        time.sleep(5)
        print("After the sleep statement")
        
        sample = wav2mfcc("./out.wav")
        sample_reshaped = sample.reshape(1,20,11,1)
        message = get_labels("./data/")[0][np.argmax(model.predict(sample_reshaped))]   

        print(message)
        mySock.send(message.encode())
        data = mySock.recv(1024).decode()
        print ("Received from server: " + data)
        


mySock.close()

* recording
* done recording


NameError: name 'wav' is not defined